In [ ]:
!pip install -U g4f

In [ ]:
import pandas as pd
from g4f.client import Client
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv("PATH_TO_DATASET")
text_column = "Sentence"

In [ ]:
# Инициализация клиента
client = Client()

In [ ]:
# Настройка промпта
system_prompt = (
    "Ты — лингвист. Определи, содержит ли данный текст эвфемизм.\n"
    "Опирайся на следующее определение: эвфемизм – это слово или выражение, которое используется вместо более прямого, грубого, табуированного или эмоционально окрашенного аналога, "
    "с целью смягчить воздействие на слушателя или создать более приемлемое, социально или культурно удобное представление о предмете, явлении или действии.\n"
    "Эвфемизмы помогают говорить о сложных, нежелательных или пугающих вопросах без риска вызвать негативные ассоциации или нарушения социальных норм.\n"
    "Отвечай строго '1', если эвфемизм есть, или '0', если нет. Без объяснений и лишнего текста."
)

In [ ]:
few_shot = """
Предложение: "Он ушёл из жизни несколько лет назад."
Ответ: 1

Предложение: "Она не очень здорова, поэтому пропустила встречу."
Ответ: 1

Предложение: "иди в баню!)"
Ответ: 1

Предложение: "Почему же Соединенные Штаты не нанесли удар по этим странам?"
Ответ: 1

Предложение: "Некоторым самолётам на авиабазе было приказано бомбить позиции повстанцев."
Ответ: 0

Предложение: "Спустя несколько дней тренер умер в больнице"
Ответ: 0

Предложение: "В результате инвалид не может комфортно посетить туалет и даже добраться до раковины."
Ответ: 0

Предложение: "Серьезной и повсеместной проблемой Афганистана остаются наркотики."
Ответ: 0
"""

In [ ]:
def classify(sentence, with_examples=True):
    user_prompt = f'\n\nПредложение: "{sentence}"\nОтвет:'
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": (few_shot if with_examples else "") + user_prompt}
    ]
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )
        content = response.choices[0].message.content.strip()
        print(f"[{sentence}] => {content}")
        if "1" in content:
            return 1
        elif "0" in content:
            return 0
        else:
            return None
    except Exception as e:
        print(f"Ошибка при обработке '{sentence}': {e}")
        return None

In [ ]:
# Тест
print(classify("Он ушёл в лучший мир."))

In [ ]:
# Классификация
df["label_zero_shot"] = df[text_column].apply(lambda x: classify(x, with_examples=False))
df["label_few_shot"] = df[text_column].apply(lambda x: classify(x, with_examples=True))

In [ ]:
df.head()

In [ ]:
from sklearn.metrics import f1_score

# Фильтруем пустые строки, если есть
df_filtered_zero_shot = df.dropna(subset=['True label', 'label_zero_shot']).copy()
df_filtered_few_shot = df.dropna(subset=['True label', 'label_few_shot']).copy()

# F1 в целом
f1_zero_overall = f1_score(df_filtered_zero_shot['True label'], df_filtered_zero_shot['label_zero_shot'], average='binary')
f1_few_overall = f1_score(df_filtered_few_shot['True label'], df_filtered_few_shot['label_few_shot'], average='binary')

# F1 по классам
f1_zero_per_class = f1_score(df_filtered_zero_shot['True label'], df_filtered_zero_shot['label_zero_shot'], average=None, labels=[0, 1])
f1_few_per_class = f1_score(df_filtered_few_shot['True label'], df_filtered_few_shot['label_few_shot'], average=None, labels=[0, 1])

print(f"F1 Zero-shot (общая): {f1_zero_overall:.4f}")
print(f"F1 Few-shot (общая): {f1_few_overall:.4f}")
print()
print(f"F1 Zero-shot — класс 0 (не эвфемизм): {f1_zero_per_class[0]:.4f}")
print(f"F1 Zero-shot — класс 1 (эвфемизм):    {f1_zero_per_class[1]:.4f}")
print()
print(f"F1 Few-shot — класс 0 (не эвфемизм):  {f1_few_per_class[0]:.4f}")
print(f"F1 Few-shot — класс 1 (эвфемизм):     {f1_few_per_class[1]:.4f}")

In [ ]:
df.to_csv("LLM_experiment.csv", index=False)